In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

In [2]:
from pyEulerCurves.distance_utils import *

In [3]:
%load_ext memory_profiler

## 1d

In [ ]:
rng = np.random.default_rng(seed=42)
N = 100000
M = 9999

contributions_1d = [
    (tuple(rng.integers(low=0, high=M, size=1)), rng.integers(low=-5, high=5))
    for _ in range(N)
]

rng2 = np.random.default_rng(seed=66)
new_contributions_1d = [
    (tuple(rng.integers(low=0, high=M, size=1)), rng.integers(low=-5, high=5))
    for _ in range(N)
]

# uncomment this for some easier cases
# contributions_1d += [((50,), 1)]

# new_contributions_1d = contributions_1d + [((50,), 3)]

In [ ]:
from pyEulerCurves.ecc_utils import difference_ECC, euler_characteristic_list_from_all

In [ ]:
%%time
d1 = difference_ECC(
        euler_characteristic_list_from_all(sorted([(f[0], c) for f, c in contributions_1d])), 
        euler_characteristic_list_from_all(sorted([(f[0], c) for f, c in new_contributions_1d])), 
        M
    )
print(d1)

In [ ]:
%%time
d2 = difference_ECP_nd(
        contributions_1d, new_contributions_1d, [(0, M)], verbose=True
    )
print(d2)

In [ ]:
assert d1 == d2

## 2d

In [4]:
rng = np.random.default_rng(seed=42)
N = 10
M = 99

contributions_2d = [
    (tuple(rng.integers(low=0, high=M, size=2)), rng.integers(low=-5, high=5))
    for _ in range(N)
]

# rng2 = np.random.default_rng(seed=66)
# new_contributions_2d = [
#     (tuple(rng.integers(low=0, high=M, size=2)), rng.integers(low=-5, high=5))
#     for _ in range(N)
# ]

new_contributions_2d = contributions_2d + [[(99, 99), 1]]

# Save to file
for i, ecp in enumerate([contributions_2d, new_contributions_2d]):
    with open("data/testECP/2d_{}".format(i), "w") as f:
        for (x, y), val in ecp:
            f.write(f"{x} {y} {val}\n")

In [5]:
%%time
d1 = difference_ECP_naive(
        contributions_2d, new_contributions_2d, tuple([(0, M+1) for _ in range(2)]), verbose=True
    )
print(d1)

computing L1 distance in the range (0, 0) - (100, 100)
1.0
CPU times: user 258 μs, sys: 21 μs, total: 279 μs
Wall time: 313 μs


In [6]:
%%time
d2 = difference_ECP(
        contributions_2d, new_contributions_2d, tuple([(0, M+1) for _ in range(2)]), verbose=True
    )
print(d2)

using optimized numba 2d
computing L1 distance in the range (0, 0) - (100, 100)
creating ECP matrix of size (3, 3)
1
CPU times: user 277 ms, sys: 20.5 ms, total: 297 ms
Wall time: 316 ms


In [7]:
assert d1 == d2

In [8]:
# %%memit
# difference_ECP_naive(
#         contributions_2d, new_contributions_2d, tuple([(0,  M+1) for _ in range(2)]), verbose=True
#     )

# %%memit
# difference_ECP_nd(
#         contributions_2d, new_contributions_2d, tuple([(0,  M+1) for _ in range(2)]), verbose=True
#     )

In [21]:
difference_ECP(
    discretize_contributions(
        contributions_2d, dims=((44.0, 100.0), (0.0, 100.0)), resolution=(30, 30)
    ),
    discretize_contributions(
        new_contributions_2d, dims=((44.0, 100.0), (0.0, 100.0)), resolution=(30, 30)
    ),
    dims=((0.0, 100.0), (0.0, 100.0)),
    verbose=True,
)

[array([44.        , 45.86666667, 47.73333333, 49.6       , 51.46666667,
       53.33333333, 55.2       , 57.06666667, 58.93333333, 60.8       ,
       62.66666667, 64.53333333, 66.4       , 68.26666667, 70.13333333,
       72.        , 73.86666667, 75.73333333, 77.6       , 79.46666667,
       81.33333333, 83.2       , 85.06666667, 86.93333333, 88.8       ,
       90.66666667, 92.53333333, 94.4       , 96.26666667, 98.13333333]), array([ 0.        ,  3.33333333,  6.66666667, 10.        , 13.33333333,
       16.66666667, 20.        , 23.33333333, 26.66666667, 30.        ,
       33.33333333, 36.66666667, 40.        , 43.33333333, 46.66666667,
       50.        , 53.33333333, 56.66666667, 60.        , 63.33333333,
       66.66666667, 70.        , 73.33333333, 76.66666667, 80.        ,
       83.33333333, 86.66666667, 90.        , 93.33333333, 96.66666667])]
value 8 goes to id -1 - 98.13333333333333
value 76 goes to id 22 - 73.33333333333334
value 43 goes to id -1 - 98.13333333333333
val

6.222222222222238

In [17]:
# compile the code, modify ecp_2d.cpp to change the limits and grid size
! g++ -std=c++11 -O2 -o ecp_dist ecp_2d.cpp

In [18]:
# run
! time ./ecp_dist data/testECP/2d_0 data/testECP/2d_1

Combined bounding box: xmin = 44, xmax = 100, ymin = 0, ymax = 100, x_step = 30, y_step = 30
Cell edges:
44 45.8667 47.7333 49.6 51.4667 53.3333 55.2 57.0667 58.9333 60.8 62.6667 64.5333 66.4 68.2667 70.1333 72 73.8667 75.7333 77.6 79.4667 81.3333 83.2 85.0667 86.9333 88.8 90.6667 92.5333 94.4 96.2667 98.1333 
Cell edges:
44 45.8667 47.7333 49.6 51.4667 53.3333 55.2 57.0667 58.9333 60.8 62.6667 64.5333 66.4 68.2667 70.1333 72 73.8667 75.7333 77.6 79.4667 81.3333 83.2 85.0667 86.9333 88.8 90.6667 92.5333 94.4 96.2667 98.1333 
L1 distance = 6.22
./ecp_dist data/testECP/2d_0 data/testECP/2d_1  0.00s user 0.00s system 1% cpu 0.148 total


## 3d

In [ ]:
rng = np.random.default_rng(seed=42)
N = 100

contributions_3d = [
    (tuple(rng.integers(low=0, high=99, size=3)), rng.integers(low=-5, high=5))
    for _ in range(N)
]

rng2 = np.random.default_rng(seed=66)
new_contributions_3d = [
    (tuple(rng.integers(low=0, high=99, size=3)), rng.integers(low=-5, high=5))
    for _ in range(N)
]

In [ ]:
%%time
d1 = difference_ECP_naive(
        contributions_3d, new_contributions_3d, tuple([(0, 100) for _ in range(3)]), verbose=True
    )
print(d1)

In [ ]:
%%time
d2 = difference_ECP_nd(
        contributions_3d, new_contributions_3d, tuple([(0, 100) for _ in range(3)]), verbose=True
    )
print(d2)

In [ ]:
%memit difference_ECP_nd(contributions_3d, new_contributions_3d, tuple([(0, 100) for _ in range(3)]), verbose=True)

In [ ]:
%memit difference_ECP_naive(contributions_3d, new_contributions_3d, tuple([(0, 100) for _ in range(3)]), verbose=True)

In [ ]:
assert d1 == d2

## 4d

In [ ]:
rng = np.random.default_rng(seed=42)
N = 50

contributions_4d = [
    (tuple(rng.integers(low=0, high=99, size=4)), rng.integers(low=-5, high=5))
    for _ in range(N)
]

rng2 = np.random.default_rng(seed=66)
new_contributions_4d = [
    (tuple(rng.integers(low=0, high=99, size=4)), rng.integers(low=-5, high=5))
    for _ in range(N)
]

In [ ]:
%%time
d1 = difference_ECP_naive(
        contributions_4d, new_contributions_4d, tuple([(0, 100) for _ in range(4)]), verbose=True
    )
print(d1)

In [ ]:
%%time
d2 = difference_ECP_nd(
        contributions_4d, new_contributions_4d, tuple([(0, 100) for _ in range(4)]), verbose=True
    )
print(d2)

In [ ]:
assert d1 == d2

In [ ]:
%memit difference_ECP_nd( contributions_4d, new_contributions_4d, tuple([(0, 100) for _ in range(4)]), verbose=True)

In [ ]:
%memit difference_ECP_naive( contributions_4d, new_contributions_4d, tuple([(0, 100) for _ in range(4)]), verbose=True)